In [1]:
import pandas as pd
import numpy as np
import os
import re

In [6]:
date = "20210709_204655"
directory = f'_output/hedges/pnl/{date}'

In [7]:
PNL = "PNL__KDE__DeltaGammaHedge__BULLISH__3000__30.csv"

In [8]:
prices = pd.read_csv("_output/hedges/pnl/20210709_204655/__prices.csv")
prices = dict(prices[["label", "price"]].to_numpy())

In [40]:
rows = []

# stats = ['mean', 'std', 'skew', 'kurt', '1%', '25%', '50%', '75%', '99%']
stats = ['mean', 'std', 'skew', 'kurt', '1%', '99%']

for fname in [fname for fname in os.listdir(directory) if fname.endswith('.csv') and fname.startswith('PNL')]:
    paths, model, hedge, period, strike, maturity = re.match("PNL__(\w+)__(\w+)__(\w+)__(\w+)__(\w+)__(\w+)__100000.csv", fname).groups()
    df = pd.read_csv(f"{directory}/{fname}", header=None)
    label = re.match(".*_(\w+__\d+__\d+)__\d+.csv$", fname).groups()[0]
    price = prices[label]
    quantile_low = df.quantile(0.01)
    quantile = df.quantile(.99)
    skew = df.skew()
    kurt = df.kurtosis()
    df = df.describe()
    df = df.T
    df['skew'] = skew
    df['kurt'] = kurt
    df['1%'] = quantile_low
    df['99%'] = quantile
    df = np.round(df/price, 3)
    del df['count']
    del df['min']
    del df['max']
    df = df[stats]
    rows += [[paths, model, hedge, period, strike, maturity, *df.to_numpy()[0]]]
    
columns = ['paths', 'model', 'hedge', 'period', 'strike', 'maturity', *stats]
df = pd.DataFrame(rows, columns=columns)

# df = df[df['maturity'] == '90']
df = df[df['strike'].isin(['3000', '7000', '9000'])]
df

,paths,model,hedge,period,strike,maturity,mean,std,skew,kurt,1%,99%
0,KDE,SVCJ,DeltaHedge,COVID,9000,30,0.105,0.180,-0.000,0.001,-0.372,0.527
1,KDE,MERTON,DeltaVegaHedge,BULLISH,3000,90,0.054,0.372,0.001,0.003,-0.842,1.208
2,SVCJ,BLACK_SCHOLES,DeltaGammaHedge,BULLISH,3000,90,0.013,0.113,-0.000,0.069,-0.310,0.293
3,KDE,SVCJ,DeltaVegaHedge,CALM,7000,90,0.038,0.076,0.000,0.001,-0.148,0.237
4,SVCJ,SVJ,DeltaHedge,CALM,7000,30,-0.004,0.377,-0.001,0.004,-1.408,0.686
...,...,...,...,...,...,...,...,...,...,...,...,...
821,SVCJ,MERTON,DeltaVegaHedge,CALM,9000,30,0.010,0.820,0.008,0.109,-1.990,2.404
824,KDE,SVCJ,MinimumVarianceHedge,CALM,9000,90,0.559,0.184,0.000,0.000,0.201,1.045
825,KDE,SVJ,DeltaGammaHedge,CALM,7000,30,0.030,0.155,0.000,0.001,-0.307,0.450
826,KDE,MERTON,MinimumVarianceHedge,CALM,9000,90,0.527,0.525,-0.000,-0.001,-0.634,1.545


In [42]:
hs = {
    'DeltaHedge': 0,
    'DeltaGammaHedge': 1,
    'DeltaVegaHedge': 2,
    'MinimumVarianceHedge': 3,
}

ms = {
    'BLACK_SCHOLES': 0,
    'HESTON': 1,
    'VARIANCE_GAMMA': 2,
    'SVJ': 3,
    'SVCJ': 4,
    'CGMY': 5,
    'MERTON': 6,
}

hn = {
    'DeltaHedge': "\Delta ",
    'DeltaGammaHedge': "\Delta-\Gamma ",
    'DeltaVegaHedge': "\Delta-\mathcal V ",
    'MinimumVarianceHedge': "\textbf{MV} ",
}

mn = {
    'BLACK_SCHOLES': "BS",
    'HESTON': "SV",
    'VARIANCE_GAMMA': "VG",
    'SVJ': 'SVJ',
    'SVCJ': 'SVCJ',
    'CGMY': 'CGMY',
    'MERTON': 'Merton',
}

for (paths, period), odf in df.groupby(['paths', 'period']):
    print("# ", paths, period, "\n\n")
    odf1 = odf[odf['maturity'] == '30']
    odf2 = odf[odf['maturity'] == '90']
    odf = pd.merge(odf1, odf2, on=['paths', 'model', 'hedge', 'period', 'strike'], suffixes=('_30', '_90'))
    odf['hs'] = odf.apply(lambda x: hs[x['hedge']], axis=1)
    odf['ms'] = odf.apply(lambda x: ms[x['model']], axis=1)
    odf['name'] = odf.apply(lambda x: f"${hn[x['hedge']]}_{{{mn[x['model']]}}}$", axis=1)
    odf = np.round(odf, 3)
    odf = odf.sort_values(['hs', 'ms'])[['name', *[f'{x}_30' for x in stats], *[f'{x}_90' for x in stats]]]
#     odf.columns = ['name', 'mean', 'std', 'skew', 'kurt', '1\%', '25\%', '50\%', '75\%', '99\%']
    caption = f"Summary statistics of {period} {paths} hedges"
    label = f"tab:{paths}_{period}"
    print(f"\\begin{{table}}\n\centering\n{odf.to_latex(index=False, escape=False)}\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\end{{table}}")
    print('\n==================================================================\n\n')

#  KDE BULLISH 


\begin{table}
\centering
\begin{tabular}{lrrrrrrrrrrrr}
\toprule
                          name &  mean_30 &  std_30 &  skew_30 &  kurt_30 &  1%_30 &  99%_30 &  mean_90 &  std_90 &  skew_90 &  kurt_90 &  1%_90 &  99%_90 \\
\midrule
                $\Delta _{BS}$ &    0.011 &   0.120 &      0.0 &    0.001 & -0.278 &   0.361 &    0.015 &   0.372 &    0.001 &    0.003 & -0.887 &   1.162 \\
                $\Delta _{SV}$ &    0.010 &   0.120 &      0.0 &    0.001 & -0.281 &   0.361 &    0.015 &   0.373 &    0.001 &    0.003 & -0.887 &   1.163 \\
                $\Delta _{VG}$ &    0.010 &   0.120 &      0.0 &    0.001 & -0.279 &   0.361 &    0.014 &   0.372 &    0.001 &    0.003 & -0.889 &   1.162 \\
               $\Delta _{SVJ}$ &    0.010 &   0.120 &      0.0 &    0.001 & -0.280 &   0.361 &    0.015 &   0.373 &    0.001 &    0.003 & -0.888 &   1.163 \\
              $\Delta _{SVCJ}$ &    0.010 &   0.120 &      0.0 &    0.001 & -0.281 &   0.361 &    0.016 &   0.373 &   

In [19]:
for fname in [fname for fname in os.listdir(".") if fname.endswith('4000__30__100000.csv') and fname.startswith('PNL__SVCJ__BLACK_SCHOLES')]:
    df = pd.read_csv(f"{fname}", header=None)
    label = re.match(".*_(\w+__\d+__\d+)__\d+.csv$", fname).groups()[0]
    price = prices[label]
    print(price)
    quantile_low = df.quantile(0.01)
    quantile = df.quantile(.99)
    skew = df.skew()
    kurt = df.kurtosis()
    df = df.describe()
    df = df.T
    df['skew'] = skew
    df['kurt'] = kurt
    df['1 %'] = quantile_low
    df['99 %'] = quantile
    df = np.round(df/price, 4)
    del df['count']
    del df['min']
    del df['max']
    df = df[['mean', 'std', 'skew', 'kurt','1 %','25%','50%','75%','99 %']]
    print(fname)
    display(df)
    print("\n\n")
    print(df.to_latex())
    print("\n\n")

201.9261905072699
PNL__SVCJ__BLACK_SCHOLES__DeltaVegaHedge__BULLISH__4000__30__100000.csv


,mean,std,skew,kurt,1 %,25%,50%,75%,99 %
0,0.0277,0.5132,0.0186,0.6715,-1.418,-0.0662,0.0803,0.2165,1.0





\begin{tabular}{lrrrrrrrrr}
\toprule
{} &    mean &     std &    skew &    kurt &    1 \% &     25\% &     50\% &     75\% &  99 \% \\
\midrule
0 &  0.0277 &  0.5132 &  0.0186 &  0.6715 & -1.418 & -0.0662 &  0.0803 &  0.2165 &   1.0 \\
\bottomrule
\end{tabular}




201.9261905072699
PNL__SVCJ__BLACK_SCHOLES__DeltaHedge__BULLISH__4000__30__100000.csv


,mean,std,skew,kurt,1 %,25%,50%,75%,99 %
0,0.103,0.567,-0.0122,0.2019,-1.8879,-0.0542,0.1501,0.3695,1.0928





\begin{tabular}{lrrrrrrrrr}
\toprule
{} &   mean &    std &    skew &    kurt &     1 \% &     25\% &     50\% &     75\% &    99 \% \\
\midrule
0 &  0.103 &  0.567 & -0.0122 &  0.2019 & -1.8879 & -0.0542 &  0.1501 &  0.3695 &  1.0928 \\
\bottomrule
\end{tabular}




201.9261905072699
PNL__SVCJ__BLACK_SCHOLES__DeltaGammaHedge__BULLISH__4000__30__100000.csv


,mean,std,skew,kurt,1 %,25%,50%,75%,99 %
0,0.0279,0.5143,0.0189,0.6854,-1.4177,-0.066,0.0808,0.2167,1.0001





\begin{tabular}{lrrrrrrrrr}
\toprule
{} &    mean &     std &    skew &    kurt &     1 \% &    25\% &     50\% &     75\% &    99 \% \\
\midrule
0 &  0.0279 &  0.5143 &  0.0189 &  0.6854 & -1.4177 & -0.066 &  0.0808 &  0.2167 &  1.0001 \\
\bottomrule
\end{tabular}






In [20]:
os.getcwd()

'/Users/lilimatic/chris/chris/_output/hedges/pnl/20210228_021905'

In [21]:
df

,mean,std,skew,kurt,1 %,25%,50%,75%,99 %
0,0.0279,0.5143,0.0189,0.6854,-1.4177,-0.066,0.0808,0.2167,1.0001
